In [ ]:
import pandas as pd
from statsforecast.core import StatsForecast
from statsforecast.models import AutoARIMA, Naive
from hierarchicalforecast.core import HierarchicalReconciliation
from hierarchicalforecast.methods import BottomUp, TopDown, MiddleOut
from pathlib import Path

# ----------------------------------------
# 1) 데이터 로드 및 기본 전처리
# ----------------------------------------
# Feather 파일에서 불러오기
feat_path = Path("data/feature_store/daily_features_per.feather")
df = pd.read_feather(feat_path)


# 사용 가능한 컬럼 목록 (참고용)
static_cats = ["spot_id", "vehicle_type", "region_code"]
time_known_cats = ["dow","week","month","quarter","is_weekend","is_holiday"]
time_known_reals = ["offday_run","rain_mm_lag1","RH","W",
                    "inventory_est","has_inventory","avg_vehicle_age_inv","avg_standard_rate_inv"]
time_unknown_reals = ["lag_1","lag_7","lag_28",
                      "roll_mean_7","roll_std_7","utilization_7",
                      "vehicle_count","coupon_count","coupon_flag_prev1",
                      "coupon_count_lag7","coupon_per_vehicle",
                      "region_total_rev","region_vehicle_count","region_rev_per_vehicle",
                      "region_vehicle_mean_rev"]
target_col = "rev_per_vehicle"

In [ ]:
# 1.1 날짜 컬럼(ds)와 타깃(y)으로 이름 변경
df = df.rename(columns={'date':'ds', target_col:'y'})

# 1.2 unique_id 생성: region_code/spot_id/vehicle_type 구조
df['unique_id'] = (
    df['region_code'].astype(str) + '/'
  + df['spot_id'].astype(str)    + '/'
  + df['vehicle_type'].astype(str)
)

# 1.3 모델에 사용할 최소 컬럼만 남기기
df_model = df[['unique_id', 'ds', 'y']].copy()